In [ ]:
# !pip install git+https://github.com/alec-tschantz/pybrid.git

In [1]:
import logging
import torch
from pybrid.scripts import main
from pybrid.config import cfg

In [4]:
cfg.exp.log_dir = "results/hybrid"

seeds = [0, 1, 2]
for seed in seeds:
    cfg.exp.seed = seed
    main(cfg)

2021-03-22 14:53:12,047 [INFO] Starting experiment @ results/hybrid/0 [cpu]
{'data': {'label_scale': 0.94,
          'normalize': True,
          'test_size': 100,
          'train_size': 1000},
 'exp': {'img_dir': 'results/hybrid/0/imgs',
         'log_dir': 'results/hybrid/0',
         'num_epochs': 20,
         'seed': 0,
         'test_every': 1},
 'infer': {'fixed_preds_test': False,
           'fixed_preds_train': False,
           'init_std': 0.01,
           'mu_dt': 0.01,
           'num_test_iters': 200,
           'num_train_iters': 50,
           'test_thresh': None,
           'train_thresh': 0.001},
 'model': {'act_fn': 'tanh',
           'amort_nodes': [784, 500, 500, 10],
           'kaiming_init': False,
           'nodes': [10, 500, 500, 784],
           'train_amortised': True,
           'use_bias': True},
 'optim': {'amort_lr': 0.0001,
           'batch_scale': True,
           'batch_size': 64,
           'grad_clip': 5,
           'lr': 0.0001,
           'name':

KeyboardInterrupt: 